In [3]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
sem_df = pd.read_csv('../data/semmedVER30_A_clean.csv')

/home/mmayers/software/anaconda3/envs/ml/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print('Rows: {:,}'.format(sem_df.shape[0]))
print('Cols: {}'.format(sem_df.shape[1]))

Rows: 97,118,423
Cols: 12


In [6]:
sem_df.head()

,PREDICATION_ID,SENTENCE_ID,PMID,PREDICATE,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE,SUBJECT_NOVELTY,OBJECT_CUI,OBJECT_NAME,OBJECT_SEMTYPE,OBJECT_NOVELTY
0,61,128,14420371,ISA,C0026879,Mutagens,hops,1,C0220806,Chemicals,chem,1
1,62,116,5659339,PART_OF,C0017725,Glucose,bacs,1,C0022378,jejunum,bpoc,1
2,63,146,12255310,PROCESS_OF,C0006147,Breast Feeding,orgf,1,C0020114,Human,humn,1
3,64,170,12305488,TREATS,C0279494,Oestrogen therapy,topp,1,C0043210,Woman,popg,1
4,65,116,5659339,PROCESS_OF,C0232338,Blood flow,orgf,1,C0012984,Canis familiaris,mamm,1


## De-Duplicate

The data will be faster to work with if duplicate items are removed.

In [7]:
sem_df = sem_df.drop_duplicates(subset=['SUBJECT_CUI', 'SUBJECT_NAME', 'SUBJECT_SEMTYPE', 'PREDICATE',
                                        'OBJECT_CUI', 'OBJECT_NAME', 'OBJECT_SEMTYPE'])

print('Rows Remaining: {:,}'.format(sem_df.shape[0]))

Rows Remaining: 21,447,153


# Sem-Types

Semtypes are abbreviated, so need to get mappings to true name.  
Examining the length of the map file, there should be 133 semtypes.

In [8]:
len(set(sem_df['SUBJECT_SEMTYPE'].unique()).union(set(sem_df['OBJECT_SEMTYPE'].unique())))

134

## Normalizing semtypes for CUIs with multiple semtypes

We know that there are multiple Semtypes for a given CUI found throghout the database.  Can we just use a voting system? That with the most occruances of a given semtype gets that semtype?

For a CUI thats a Gene e.g. `C1337111` or the `POMC gene` we want something thats a subclass of Genes and Genomes (aka gngm).

In [9]:
sem_df.query('SUBJECT_CUI == "C1337111"')['SUBJECT_SEMTYPE'].value_counts()

aapp    4361
gngm     926
horm     904
phsu     292
nsba      32
bacs       1
Name: SUBJECT_SEMTYPE, dtype: int64

Unfortunately, aapp is a sublcass of Chemicals & Drugs, so this would be given the wrong superclass if identified.

However, it will be easier to do this if the nodes are sperated from the edges first


### Semmtypes abbreviation mappings

Files with the mapping information were [downloaded here](https://metamap.nlm.nih.gov/SemanticTypesAndGroups.shtml)

In [10]:
abbv_to_type = dict()
with open('../SemanticTypes_2013AA.txt') as fin:
    for line in fin:
        line = line.strip()
        lspt = line.split('|')
        abbv_to_type[lspt[0]] = lspt[-1]
type_to_abbv = {v:k for k, v in abbv_to_type.items()}

Semtypes also have a supertype.   
The mapping file has this structure:

    ANAT|Anatomy|T017|Anatomical Structure

### Semtype supertypes mappings

In [11]:
abbv_to_super = dict()
with open('../SemGroups_2013.txt') as fin:
    for line in fin:
        line = line.strip()
        lspt = line.split('|')
        abbv_to_super[type_to_abbv[lspt[-1]]] = lspt[1]

### Check to see that all of the semtype abbreviations in the SemmedDB are repepresented in this mapping file

In [12]:
semtype_abbvs = abbv_to_super.keys()
unused_abbvs = set(sem_df.query('SUBJECT_SEMTYPE not in @semtype_abbvs')['SUBJECT_SEMTYPE'].unique())
unused_abbvs.update(set(sem_df.query('OBJECT_SEMTYPE not in @semtype_abbvs')['OBJECT_SEMTYPE'].unique()))

unused_abbvs

{'0', 'alga', 'invt', 'rich'}

There are a few that aren't in there (ignore the 0, thats probably a corrupted line), so lets see if we can figure out what they should be

In [13]:
sem_df.query('SUBJECT_SEMTYPE == "alga"').head(2)[['SUBJECT_CUI', 'SUBJECT_NAME', 'SUBJECT_SEMTYPE']]

,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE
2807,C0002032,"Algae, Green",alga
5032,C0002028,Algae,alga


In [14]:
sem_df.query('SUBJECT_SEMTYPE == "invt"').head(2)[['SUBJECT_CUI', 'SUBJECT_NAME', 'SUBJECT_SEMTYPE']]

,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE
197,C0012518,Dipetalonema perstans,invt
302,C0026584,Culicidae,invt


In [15]:
sem_df.query('SUBJECT_SEMTYPE == "rich"').head(2)[['SUBJECT_CUI', 'SUBJECT_NAME', 'SUBJECT_SEMTYPE']]

,SUBJECT_CUI,SUBJECT_NAME,SUBJECT_SEMTYPE
1623,C0035587,Rickettsia prowazekii,rich
6752,C0010240,Coxiella burnetii,rich


In [16]:
# Looked ups a few cuis with these and they are all of the living beings type, 
# so we will set these to that semtype
abbv_to_super['invt'] = 'Living Beings'
abbv_to_super['alga'] = 'Living Beings'
abbv_to_super['rich'] = 'Living Beings'

In [17]:
# Make a nodes csv

snodes = pd.DataFrame()
onodes = pd.DataFrame()


snodes['ID'] = sem_df['SUBJECT_CUI']
snodes['name'] = sem_df['SUBJECT_NAME']
snodes['LABEL'] = sem_df['SUBJECT_SEMTYPE'].apply(lambda x: abbv_to_super.get(x))

onodes['ID'] = sem_df['OBJECT_CUI']
onodes['name'] = sem_df['OBJECT_NAME']
onodes['LABEL'] = sem_df['OBJECT_SEMTYPE'].apply(lambda x: abbv_to_super.get(x))


nodes = pd.concat([snodes, onodes])
nodes.drop_duplicates(inplace=True)
nodes = nodes.reset_index(drop=True)

nodes.head()

,ID,name,LABEL
0,C0026879,Mutagens,Chemicals & Drugs
1,C0017725,Glucose,Chemicals & Drugs
2,C0006147,Breast Feeding,Physiology
3,C0279494,Oestrogen therapy,Procedures
4,C0232338,Blood flow,Physiology


In [18]:
print('There are {} lines and {} unique IDs'.format(nodes.shape[0], nodes['ID'].nunique()))

num_duped = nodes[nodes.duplicated(subset='ID', keep=False)].drop_duplicates().shape[0]
print('{} IDs have been found to have multiple semantic types'.format(num_duped))

There are 304433 lines and 268918 unique IDs
70015 IDs have been found to have multiple semantic types


Dropping all NA values drops way too many.  We need to see which ones have NA values and why

In [19]:
no_semtype = nodes.isnull().sum(axis=1).astype(bool)
nodes[no_semtype]

,ID,name,LABEL
304124,C3146964,COX5BP4 gene,None


In [20]:
cui_to_t_code = dict()
with open('../MRSTY.RRF') as fin:
    for line in fin:
        line = line.strip()
        ls = line.split('|')
        cui_to_t_code[ls[0]] = [x for x in ls[1:] if x != '']

In [21]:
t_code_to_super = dict()
with open('../SemGroups_2013.txt') as fin:
    for line in fin:
        line = line.strip()
        ls = line.split('|')
        t_code_to_super[ls[2]] = ls[1]

In [22]:
for i, (k, v) in enumerate(cui_to_t_code.items()):
    print(k, v)
    if i==10:
        break

C2069586 ['T061']
C2655812 ['T002']
C0239651 ['T033']
C1649932 ['T015']
C1899978 ['T002']
C0104020 ['T121']
C3440209 ['T007']
C0703171 ['T200']
C2946216 ['T200']
C1142884 ['T123']
C1959698 ['T023']


In [23]:
cui_to_super = dict()
for k, v in cui_to_t_code.items():
    # Only 1 T-Code, so map to its super semmantic type
    if len(v) == 1:
        cui_to_super[k] = t_code_to_super[v[0]]
    else:
        # Most multiple T-Codes should have the same supertype
        v_set = set([t_code_to_super[x] for x in v])
        if len(v_set) == 1:
            cui_to_super[k] = v_set.pop()
        # If they don't have the same supertype, return as a list...
        else:
            # If nothing prints they we have true 1-1 mapping and noting to worry about
            print(k, v_set)
            cui_to_super[k] = list(v_set)



## Relabel the semmantic types to their true type

In [24]:
nodes['true_label'] = nodes['ID'].apply(lambda x: cui_to_super.get(x, x))

In [25]:
p_idx = nodes['true_label'].str.startswith('C')
bad_ids = set(nodes[p_idx]['true_label'])
print("{} CUIs not found for true Semmantic Mapping...".format(len(bad_ids)))

14143 CUIs not found for true Semmantic Mapping...


Some node CUIs were not mappable to a semmatic type.  We will use what semmedDB provided for a semantic type in these instances.

In [26]:
potential_problems = nodes.query('ID in @bad_ids').copy()

In [27]:
potential_problems['true_label'] = potential_problems['LABEL']
problems = potential_problems[potential_problems.duplicated(subset='ID', keep=False)].copy()
print("Now {} CUIs have multiple Semmantic Types".format(problems['ID'].nunique()))

Now 1667 CUIs have multiple Semmantic Types


In [28]:
fixed_probs = potential_problems.drop(problems.index)

### Combining Multiple semtypes for a single object

In [29]:
pairs = []
grpd = problems.groupby('ID')
for grp, df in grpd:
    pairs.append(tuple(df.sort_values('LABEL')['LABEL']))
pairs = set(pairs)
len(pairs)

12

In [30]:
pairs = sorted(list(pairs))
pairs

[('Activities & Behaviors', 'Procedures'),
 ('Chemicals & Drugs', 'Genes & Molecular Sequences'),
 ('Chemicals & Drugs', 'Objects'),
 ('Concepts & Ideas', 'Devices'),
 ('Concepts & Ideas', 'Disorders'),
 ('Concepts & Ideas', 'Living Beings'),
 ('Concepts & Ideas', 'Objects'),
 ('Concepts & Ideas', 'Procedures'),
 ('Living Beings', 'Objects'),
 ('Objects', 'Organizations'),
 ('Phenomena', 'Physiology'),
 ('Physiology', 'Procedures')]

In [31]:
def look_at_semtype_pairs(dups, pairs):
    types = dups['LABEL'] == pairs[0]
    for p in pairs[1:]:
        current_type = dups['LABEL'] == p
        types = types | current_type
        
    return dups[types][dups.loc[types, 'ID'].duplicated(keep=False)].sort_values('ID')

In [32]:
# 1 and 7 are long, most are small
idxs = list(i for i in range(12))
idxs.remove(1)
idxs.remove(7)

pd.concat([look_at_semtype_pairs(problems, pairs[i]) for i in idxs])

,ID,name,LABEL,true_label
30840,C0683532,government health service,Procedures,Procedures
276378,C0683532,government health service,Activities & Behaviors,Activities & Behaviors
79588,C0682953,saturated dietary fat,Chemicals & Drugs,Chemicals & Drugs
248273,C0682953,saturated dietary fat,Objects,Objects
85440,C0262798,Clinical Information Systems,Devices,Devices
241550,C0262798,Clinical Information Systems,Concepts & Ideas,Concepts & Ideas
58983,C0262877,Medical Information Systems,Devices,Devices
245298,C0262877,Medical Information Systems,Concepts & Ideas,Concepts & Ideas
134403,C1608931,"Information Systems, Data Management, Hospital",Devices,Devices
197788,C1608931,"Information Systems, Data Management, Hospital",Concepts & Ideas,Concepts & Ideas


In [33]:
look_at_semtype_pairs(problems, pairs[1]).head(16)

,ID,name,LABEL,true_label
277530,C0002195,alpha 1-Proteinase Inhibitor F,Genes & Molecular Sequences,Genes & Molecular Sequences
95221,C0002195,alpha 1-Proteinase Inhibitor F,Chemicals & Drugs,Chemicals & Drugs
61654,C0002242,Alpha-Amanitine,Chemicals & Drugs,Chemicals & Drugs
271208,C0002242,Alpha-Amanitine,Genes & Molecular Sequences,Genes & Molecular Sequences
244229,C0003731,ARC-1,Genes & Molecular Sequences,Genes & Molecular Sequences
115779,C0003731,ARC-1,Chemicals & Drugs,Chemicals & Drugs
48016,C0004619,Bacterial Elongation Factor Tu,Chemicals & Drugs,Chemicals & Drugs
275979,C0004619,Bacterial Elongation Factor Tu,Genes & Molecular Sequences,Genes & Molecular Sequences
119677,C0005179,beta-1 Globin,Genes & Molecular Sequences,Genes & Molecular Sequences
54785,C0005179,beta-1 Globin,Chemicals & Drugs,Chemicals & Drugs


In [34]:
look_at_semtype_pairs(problems, pairs[7]).head(16)

,ID,name,LABEL,true_label
230547,C0679579,Barron-Welsh Art Scale,Procedures,Procedures
107039,C0679579,Barron-Welsh Art Scale,Concepts & Ideas,Concepts & Ideas
89536,C0681925,logistic regression analysis,Procedures,Procedures
100499,C0681925,logistic regression analysis,Concepts & Ideas,Concepts & Ideas
84829,C0681926,multiple analysis of variance,Concepts & Ideas,Concepts & Ideas
128543,C0681926,multiple analysis of variance,Procedures,Procedures
156394,C0681927,bivariate analysis,Concepts & Ideas,Concepts & Ideas
89457,C0681927,bivariate analysis,Procedures,Procedures
203187,C0681928,canonical correlation analysis,Procedures,Procedures
81080,C0681928,canonical correlation analysis,Concepts & Ideas,Concepts & Ideas


In [35]:
semtype_hierarchy = [['Concepts & Ideas', 'Living Beings'],
                     ['Concepts & Ideas', 'Devices'],
                     ['Concepts & Ideas', 'Objects'],
                     ['Concepts & Ideas', 'Disorders'],
                     ['Physiology', 'Phenomena'], 
                     ['Physiology', 'Procedures'], 
                     ['Chemicals & Drugs', 'Objects'],
                     ['Procedures', 'Activities & Behaviors'],
                     ['Organizations', 'Objects'],
                     ['Living Beings', 'Objects'],
                     ['Procedures', 'Concepts & Ideas'],
                     ['Genes & Molecular Sequences', 'Chemicals & Drugs']]

In [36]:
def fix_multi_semtype(dups, semtypes):
    
    # Get the duplicates for the given types
    types = dups['LABEL'] == semtypes[0]
    for p in semtypes[1:]:
        current_type = dups['LABEL'] == p
        types = types | current_type
        
    # Only keep those where the ID appears multiple times
    duplicated = dups[types][dups.loc[types, 'ID'].duplicated(keep=False)]
    to_remove = duplicated.index
    duplicated = duplicated.reset_index(drop=True)
    
    
    # Switch to the preferred type
    duplicated['LABEL'] = semtypes[0]
    # Now remove the duplicates
    duplicated = duplicated.drop_duplicates()
    
    # return the fixed data
    return duplicated.reset_index(drop=True), to_remove


In [37]:
fixed_multi_types = []
for pair in semtype_hierarchy:
    fixed, remove = fix_multi_semtype(problems, pair)
    fixed_multi_types.append(fixed)
    problems.drop(remove, inplace=True)
fixed_semtype_df = pd.concat(fixed_multi_types)

In [38]:
fixed_semtype_df['true_label'] = fixed_semtype_df['LABEL']
fixed_semtype_df = fixed_semtype_df.drop_duplicates().reset_index(drop=True)

### Recombine all the nodes with the new semmantic types

In [39]:
before_len = len(nodes)
nodes = nodes.drop(potential_problems.index)
nodes = pd.concat([nodes, fixed_probs, fixed_semtype_df])
nodes['LABEL'] = nodes['true_label']
nodes = nodes[['ID', 'name', 'LABEL']].drop_duplicates().reset_index(drop=True)

print("Went from {} to {} nodes by setting semantic type".format(before_len, len(nodes)))
print("{} Unique CUIs".format(nodes['ID'].nunique()))

Went from 304433 to 269552 nodes by setting semantic type
268918 Unique CUIs


In [40]:
nodes[nodes['ID'].duplicated(keep=False)].sort_values('ID').head(10)

,ID,name,LABEL
206525,C0008288,CIPC gene,Genes & Molecular Sequences
25593,C0008288,Chlorpropham,Genes & Molecular Sequences
100814,C0017351,IGH gene,Genes & Molecular Sequences
60237,C0017351,IGH@ gene cluster,Genes & Molecular Sequences
110538,C0026574,MOS gene,Genes & Molecular Sequences
226206,C0026574,mos Oncogenes,Genes & Molecular Sequences
215206,C0026900,v-myb Genes,Genes & Molecular Sequences
129620,C0026900,MYB gene,Genes & Molecular Sequences
86510,C0035018,v-rel Genes,Genes & Molecular Sequences
108998,C0035018,REL gene,Genes & Molecular Sequences


It appears that there are many with two names for the same CUI. If we already have all the names from pinging hte UMLS API earlier, then we can just normalize them to the api results.

Lets check that all of these CUIs are in our cui_to_name mapper from previous notebook...

In [41]:
c_to_name_dict = pickle.load(open( "../data/cui_to_name.pkl", "rb" ))

ids = set(nodes[nodes['ID'].duplicated(keep=False)].sort_values('ID')['ID'].unique())
ids.issubset(set(c_to_name_dict.keys()))

True

And now just apply the new names

In [42]:
multi_names = nodes['ID'].duplicated(keep=False)
nodes.loc[multi_names, 'name'] = nodes.loc[multi_names, 'ID'].apply(lambda x: c_to_name_dict[x])
nodes = nodes.drop_duplicates().reset_index(drop=True)

In [43]:
print('There are {} nodes with {} unique IDs'.format(nodes.shape[0], nodes['ID'].nunique()))

There are 268918 nodes with 268918 unique IDs


Looks like we finally have a 1:1 relationship of unique IDs to rows in our nodes table

## Subsetting Diseases -  DOIDs

Ensure diseases in this graph are all mappable to a Disease Ontology ID

In [44]:
doid_cui = pd.read_csv('../doid-to-umls.csv')
print(len(doid_cui))
doid_cui.head()

6609


,doid,umlscui
0,DOID:8741,C0036508
1,DOID:11907,C0013568
2,DOID:9835,C0034951
3,DOID:0060263,C0302892
4,DOID:0060263,C1867983


In [45]:
disease_cuis = doid_cui['umlscui'].values

In [46]:
didx = nodes.query('ID in @disease_cuis').index
nodes.loc[didx, 'LABEL'] = 'Disease'

## Reduce the number of Compounds

Try to reduce the numbe of compounds in a similar manner

Here are the different T codes for the various Chemicals and Drugs... ones that we think may be interesting to look at as treatment compounds have been highlighted:

T116 - Amino Acid, Peptide, or Protein  
T195 - Antibiotic  
T123 - Biologically Active Substance  
T122 - Biomedical or Dental Material  
T118 - Carbohydrate  
T103 - Chemical  
T120 - Chemical Viewed Functionally  
T104 - Chemical Viewed Structurally  
**T200 - Clinical Drug**  
T111 - Eicosanoid  
T196 - Element, Ion, or Isotope  
T126 - Enzyme  
T131 - Hazardous or Poisonous Substance  
T125 - Hormone  
T129 - Immunologic Factor  
T130 - Indicator, Reagent, or Diagnostic Aid  
T197 - Inorganic Chemical  
T119 - Lipid  
T124 - Neuroreactive Substance or Biogenic Amine  
T114 - Nucleic Acid, Nucleoside, or Nucleotide  
**T109 - Organic Chemical**  
T115 - Organophosphorus Compound  
T121 - Pharmacologic Substance  
T192 - Receptor  
T110 - Steroid  
T127 - Vitamin   


In [47]:
good_types = set(['T109', 'T200'])

comps = []

for i, (k, v) in enumerate(cui_to_t_code.items()):
    if len(good_types.intersection(set(v))) > 0:
        comps.append(k)


In [48]:
comps_in_graph = list(set(nodes['ID']).intersection(set(comps)))

comps_idx = nodes.query('ID in @comps_in_graph').index

nodes.loc[comps_idx, 'LABEL'] = 'Compound'

In [49]:
nodes['LABEL'].value_counts()

Chemicals & Drugs              78321
Living Beings                  49094
Disorders                      38301
Genes & Molecular Sequences    20934
Compound                       20196
Procedures                     16415
Anatomy                        15533
Physiology                      8579
Devices                         4989
Disease                         4943
Concepts & Ideas                4711
Objects                         1832
Activities & Behaviors          1541
Phenomena                       1378
Organizations                    907
Occupations                      734
Geographic Areas                 510
Name: LABEL, dtype: int64

In [50]:
c = (nodes['LABEL'] == 'Compound').sum()
d = (nodes['LABEL'] == 'Disease').sum()

'{:,} Compounds * {:,} Diseases = {:,} C-D Pairs'.format(c, d, c*d)

'20,196 Compounds * 4,943 Diseases = 99,828,828 C-D Pairs'

# Edges File

## Prediacate types

[Semmed Docs](https://skr3.nlm.nih.gov/SemMed/jsp/SemMedDocumentation2.pdf)


The docs above show only 26 predicates, lets see how many the data actually contains an and what they are...

In [51]:
sem_df['PREDICATE'].value_counts()

LOCATION_OF             2746340
INTERACTS_WITH          1813431
PART_OF                 1808284
AFFECTS                 1765977
COEXISTS_WITH           1750633
TREATS                  1476188
STIMULATES              1074318
PROCESS_OF               908047
CAUSES                   879335
INHIBITS                 867715
ASSOCIATED_WITH          824583
AUGMENTS                 701009
USES                     615550
PRODUCES                 522216
DISRUPTS                 503144
compared_with            452265
ISA                      403766
DIAGNOSES                263966
PREDISPOSES              246934
METHOD_OF                204355
ADMINISTERED_TO          184601
PREVENTS                 182918
NEG_AFFECTS              144872
higher_than              130226
NEG_INTERACTS_WITH       130151
PRECEDES                  99986
OCCURS_IN                 65425
NEG_LOCATION_OF           61314
CONVERTS_TO               57636
NEG_COEXISTS_WITH         50412
                         ...   
MANIFEST

Looks like 62 total predicates.  There are negated predicats, but still, negating all should only yield 52 total.  There are also some that are all lower-case, which do not seem to be found in the documents.

In [56]:
sem_df['PREDICATE'].unique()

array(['ISA', 'PART_OF', 'PROCESS_OF', 'TREATS', 'LOCATION_OF',
       'METHOD_OF', 'INTERACTS_WITH', 'COEXISTS_WITH', 'DIAGNOSES',
       'CAUSES', 'AFFECTS', 'ASSOCIATED_WITH', 'USES', 'PRECEDES',
       'compared_with', 'PREVENTS', 'NEG_INHIBITS', 'ADMINISTERED_TO',
       'OCCURS_IN', 'NEG_ADMINISTERED_TO', 'STIMULATES', 'DISRUPTS',
       'COMPLICATES', 'higher_than', 'INHIBITS', 'AUGMENTS',
       'MANIFESTATION_OF', 'different_from', 'NEG_AFFECTS', 'PREDISPOSES',
       'NEG_higher_than', 'same_as', 'PRODUCES', 'NEG_DISRUPTS',
       'NEG_CAUSES', 'NEG_INTERACTS_WITH', 'NEG_PROCESS_OF',
       'NEG_COEXISTS_WITH', 'NEG_PART_OF', 'NEG_USES', 'lower_than',
       'NEG_PREVENTS', 'NEG_LOCATION_OF', 'NEG_METHOD_OF', 'CONVERTS_TO',
       'NEG_DIAGNOSES', 'NEG_AUGMENTS', 'NEG_PREDISPOSES', 'NEG_TREATS',
       'NEG_ASSOCIATED_WITH', 'NEG_PRODUCES', 'NEG_COMPLICATES',
       'NEG_STIMULATES', 'NEG_OCCURS_IN', 'different_than',
       'NEG_CONVERTS_TO', 'NEG_lower_than', 'NEG_MANIFESTA

### Abbreviations

According to rephetio conventions, edges must be written as `EDGETYPE_abbrev` where the abbreviation is START_NODE Abv, Predicate Abv, END_NODE Abv.  for example a Compound that treats a disease is abbreviated TREATS_CtD.  

We must come up with unique abbreviations for each semantic type for the nodes as well as each predicate. These concatinated together will give a unique abbreviation for each edge type.

In [57]:
p_abv = {
 'ADMINISTERED_TO': 'at',
 'AFFECTS': 'af',
 'ASSOCIATED_WITH': 'aw',
 'AUGMENTS': 'ag',
 'CAUSES': 'c',
 'COEXISTS_WITH': 'cw',
 'COMPLICATES': 'cp',
 'CONVERTS_TO': 'ct',
 'DIAGNOSES': 'dg',
 'DISRUPTS': 'ds',
 'INHIBITS': 'in',
 'INTERACTS_WITH': 'iw',
 'ISA': 'i',
 'LOCATION_OF': 'lo',
 'MANIFESTATION_OF': 'mfo',
 'METHOD_OF': 'mo',
 'NEG_ADMINISTERED_TO': 'nat',
 'NEG_AFFECTS': 'naf',
 'NEG_ASSOCIATED_WITH': 'naw',
 'NEG_AUGMENTS': 'nag',
 'NEG_CAUSES': 'nc',
 'NEG_COEXISTS_WITH': 'ncw',
 'NEG_COMPLICATES': 'ncp',
 'NEG_CONVERTS_TO': 'nct',
 'NEG_DIAGNOSES': 'ndg',
 'NEG_DISRUPTS': 'nds',
 'NEG_INHIBITS': 'nin',
 'NEG_INTERACTS_WITH': 'niw',
 'NEG_LOCATION_OF': 'nlo',
 'NEG_MANIFESTATION_OF': 'nmfo',
 'NEG_METHOD_OF': 'nmo',
 'NEG_OCCURS_IN': 'noi',
 'NEG_PART_OF': 'npo',
 'NEG_PRECEDES': 'npc',
 'NEG_PREDISPOSES': 'nps',
 'NEG_PREVENTS': 'npv',
 'NEG_PROCESS_OF': 'npro',
 'NEG_PRODUCES': 'npd',
 'NEG_STIMULATES': 'nst',
 'NEG_TREATS': 'nt',
 'NEG_USES': 'nu',
 'NEG_higher_than': 'nht',
 'NEG_lower_than': 'nlt',
 'OCCURS_IN': 'oi',
 'PART_OF': 'po',
 'PRECEDES': 'pc',
 'PREDISPOSES': 'ps',
 'PREP': 'pr',
 'PREVENTS': 'pv',
 'PROCESS_OF': 'pro',
 'PRODUCES': 'pd',
 'STIMULATES': 'st',
 'TREATS': 't',
 'USES': 'u',
 'compared_with': 'cpw',
 'different_from': 'df',
 'different_than': 'dt',
 'higher_than': 'ht',
 'lower_than': 'lt',
 'same_as': 'sa'}

In [58]:
# get abbreviations for semtypes
super_sems = set(abbv_to_super.values())

len(super_sems)
super_sems

{'Activities & Behaviors',
 'Anatomy',
 'Chemicals & Drugs',
 'Concepts & Ideas',
 'Devices',
 'Disorders',
 'Genes & Molecular Sequences',
 'Geographic Areas',
 'Living Beings',
 'Objects',
 'Occupations',
 'Organizations',
 'Phenomena',
 'Physiology',
 'Procedures'}

In [59]:
# Run these by hand as there are few, and some don't lend themselves well to auto-generation
sem_abv = {
 'Activities & Behaviors': 'AB',
 'Anatomy': 'A',
 'Compound': 'C',
 'Chemicals & Drugs': 'CD',
 'Concepts & Ideas': 'CI',
 'Devices': 'DV',
 'Disease': 'D',
 'Disorders': 'DO',
 'Genes & Molecular Sequences': 'G',
 'Geographic Areas': 'GA',
 'Living Beings': 'LB',
 'Objects': 'OB',
 'Occupations': 'OC',
 'Organizations': 'OR',
 'Phenomena': 'PH',
 'Physiology': 'PS',
 'Procedures': 'PR'
}

# Edge Directions

Most edges may be bi-directions, but some are definitely better exprested as uni-directional.  Here we will classify predicates as bi- or uni-dierectional based on the semantic meaning of the predicate.  Bidirectionl edges have an empty string, whereas uni-directional have a direction marker `'>'`. For example an undirected edge, such as a compound that treats a disease, is written as `TREATS_CtD`. However, a directed edge, like Compound1 converts to Compound 2 would be written as `CONVERTS_TO_Cct>C`.

Edge directions were chosen based on the description found in [this documnetation](https://skr3.nlm.nih.gov/SemMed/jsp/SemMedDocumentation2.pdf).

In [60]:
edge_dir = {
 'ADMINISTERED_TO': '',
 'AFFECTS': '',
 'ASSOCIATED_WITH': '',
 'AUGMENTS': '',
 'CAUSES': '',
 'COEXISTS_WITH': '',
 'COMPLICATES': '',
 'CONVERTS_TO': '>',
 'DIAGNOSES': '',
 'DISRUPTS': '',
 'INHIBITS': '',
 'INTERACTS_WITH': '',
 'ISA': '>',
 'LOCATION_OF': '',
 'MANIFESTATION_OF': '>',
 'METHOD_OF': '',
 'NOM': '',
 'OCCURS_IN': '',
 'PART_OF': '>',
 'PRECEDES': '>',
 'PREDISPOSES': '',
 'PREP': '',
 'PREVENTS': '',
 'PROCESS_OF': '>',
 'PRODUCES': '>',
 'SPEC': '',
 'STIMULATES': '',
 'TREATS': '',
 'USES': '',
 'compared_with': '',
 'different_from': '',
 'different_than': '',
 'higher_from': '',
 'higher_than': '',
 'lower_than': '',
 'same_as': '',
 'than_as': ''}


#for k in p_abv:
#    if k not in edge_dir:
#        edge_dir[k] = edge_dir[k[4:]]
#edge_dir

In [61]:
# Make a mapper from SEMTYPE to sem abbrev
kind_map = {}

for k, v in abbv_to_super.items():
    kind_map[k] = sem_abv[v]

In [62]:
id_to_type = nodes.set_index('ID')['LABEL'].to_dict()
id_to_type_abbv = {k: sem_abv.get(v) for k, v in id_to_type.items()}

In [63]:
edges = pd.DataFrame()

edges['START_ID'] = sem_df['SUBJECT_CUI']
edges['END_ID'] = sem_df['OBJECT_CUI']
edges['TYPE'] = sem_df['PREDICATE']

relation_condense_map = {
'INTERACTS_WITH': 'general_association_with',
'COEXISTS_WITH': 'general_association_with',
'ASSOCIATED_WITH': 'general_association_with',
'AFFECTS': 'general_association_with',
'METHOD_OF': 'general_association_with',
'LOCATION_OF': 'general_association_with',
'ADMINISTERED_TO': 'general_association_with',

'STIMULATES': 'positive_association', 
'CAUSES': 'positive_association', 
'AUGMENTS': 'positive_association', 
'PREDISPOSES': 'positive_association', 
'COMPLICATES': 'positive_association', 

'INHIBITS': 'negative_association', 
'TREATS': 'negative_association', 
'PREVENTS': 'negative_association', 
'DISRUPTS': 'negative_association', 

'PRODUCES': 'b_bigger_than_a',
'PROCESS_OF': 'b_bigger_than_a',
'USES': 'b_bigger_than_a',

'ISA': 'a_bigger_than_b', 
'PART_OF': 'a_bigger_than_b', 
'OCCURS_IN': 'a_bigger_than_b', 
'AFFECTS': 'a_bigger_than_b', 
'MANIFESTATION_OF': 'a_bigger_than_b', 

'PRECEDES': 'precedes', 
'DIAGNOSES': 'diagnoses', 

'compared_with':  'odd_relations', 
'different_from': 'odd_relations', 
'different_than': 'odd_relations', 
'higher_from': 'odd_relations', 
'higher_than': 'odd_relations', 
'lower_than':  'odd_relations', 
'same_as':  'odd_relations', 
'than_as':  'odd_relations', 

'NEG_INTERACTS_WITH': 'neg_general_association_with',
'NEG_COEXISTS_WITH': 'neg_general_association_with',
'NEG_ASSOCIATED_WITH': 'neg_general_association_with',
'NEG_AFFECTS': 'neg_general_association_with',
'NEG_METHOD_OF': 'neg_general_association_with',
'NEG_LOCATION_OF': 'neg_general_association_with',
'NEG_ADMINISTERED_TO': 'neg_general_association_with',
'NEG_STIMULATES': 'neg_positive_association', 
'NEG_CAUSES': 'neg_positive_association', 
'NEG_AUGMENTS': 'neg_positive_association', 
'NEG_PREDISPOSES': 'neg_positive_association', 
'NEG_COMPLICATES': 'neg_positive_association', 
'NEG_INHIBITS': 'neg_negative_association', 
'NEG_TREATS': 'neg_negative_association', 
'NEG_PREVENTS': 'neg_negative_association', 
'NEG_DISRUPTS': 'neg_negative_association', 
'NEG_PRODUCES': 'neg_b_bigger_than_a',
'NEG_PROCESS_OF': 'neg_b_bigger_than_a',
'NEG_USES': 'neg_b_bigger_than_a',
'NEG_ISA': 'neg_a_bigger_than_b', 
'NEG_PART_OF': 'neg_a_bigger_than_b', 
'NEG_OCCURS_IN': 'neg_a_bigger_than_b', 
'NEG_AFFECTS': 'neg_a_bigger_than_b', 
'NEG_MANIFESTATION_OF': 'neg_a_bigger_than_b', 
'NEG_PRECEDES': 'neg_precedes', 
'NEG_DIAGNOSES': 'neg_diagnoses', 

'compared_with':  'neg_odd_relations', 
'different_from': 'neg_odd_relations', 
'different_than': 'neg_odd_relations', 
'higher_from': 'neg_odd_relations', 
'higher_than': 'neg_odd_relations', 
'lower_than':  'neg_odd_relations', 
'same_as':  'neg_odd_relations', 
'than_as':  'neg_odd_relations'}

edge_dir = {
 'a_bigger_than_b': '>',
 'b_bigger_than_a': '>',
 'diagnoses': '',
 'general_association_with': '',
 'neg_a_bigger_than_b': '>',
 'neg_b_bigger_than_a': '>',
 'neg_diagnoses': '',
 'neg_general_association_with': '',
 'neg_negative_association': '',
 'neg_odd_relations': '',
 'neg_positive_association': '',
 'neg_precedes': '>',
 'negative_association': '',
 'positive_association': '',
 'precedes': '>'}

p_abv = {
 'a_bigger_than_b': 'ab',
 'b_bigger_than_a': 'ba',
 'diagnoses': 'd',
 'general_association_with': 'a',
 'neg_a_bigger_than_b': 'nab',
 'neg_b_bigger_than_a': 'nba',
 'neg_diagnoses': 'nd',
 'neg_general_association_with': 'na',
 'neg_negative_association': 'nn',
 'neg_odd_relations': 'nor',
 'neg_positive_association': 'np',
 'neg_precedes': 'pr',
 'negative_association': 'n',
 'positive_association': 'p',
 'precedes': 'pr'}

In [64]:
print(len(edges))
edges = edges.query('TYPE in @p_abv.keys()')
print(len(edges))

21447153
21447133


In [65]:
edges['abbrev'] = (edges['START_ID'].apply(lambda x: id_to_type_abbv.get(x,x)) 
                    + edges['TYPE'].apply(lambda x: p_abv.get(x,x))
                    + edges['TYPE'].apply(lambda x: edge_dir.get(x,''))
                    + edges['END_ID'].apply(lambda x: id_to_type_abbv.get(x,x)))

edges['rev_abbrev'] = (edges['END_ID'].apply(lambda x: id_to_type_abbv.get(x,x)) 
                    + edges['TYPE'].apply(lambda x: edge_dir.get(x,''))
                    + edges['TYPE'].apply(lambda x: p_abv.get(x,x))
                    + edges['START_ID'].apply(lambda x: id_to_type_abbv.get(x,x)))


print(edges.shape[0])
edges = edges.drop_duplicates(subset=['START_ID', 'END_ID', 'TYPE'])
print(len(edges))
edges.head()

21447133
20891064


,START_ID,END_ID,TYPE,abbrev,rev_abbrev
0,C0026879,C0220806,ISA,CDi>CD,CD>iCD
1,C0017725,C0022378,PART_OF,CDpo>A,A>poCD
2,C0006147,C0020114,PROCESS_OF,PSpro>LB,LB>proPS
3,C0279494,C0043210,TREATS,PRtLB,LBtPR
4,C0232338,C0012984,PROCESS_OF,PSpro>LB,LB>proPS


In [66]:
def validate_abbrevs(e_type, abbrev, rev_abbrev):
    if rev_abbrev not in abbreviations:
        abbreviations.update([abbrev])
        return str(e_type) + '_' + str(abbrev)
    else:
        return str(e_type) + '_' + str(rev_abbrev)

In [ ]:
#### Keeping for Legacy.... This shows how poor df.apply(f, axis=1) performance is in comparison to df.itertuples()
# %%time
# abbreviations = set()
# edges['TYPE'] = edges.apply(lambda row: validate_abbrevs(row['TYPE'], row['abbrev'], row['rev_abbrev']), axis=1)
# CPU times: user 21min 2s, sys: 776 ms, total: 21min 3s
# Wall time: 21min 3s

    CPU times: user 21min 2s, sys: 776 ms, total: 21min 3s
    Wall time: 21min 3s

In [67]:
"df.apply(f,axis=1) is {:1.1f}x slower than df.itertuples() on a df of almost 21M rows".format((21*60+2) / (1*60+29))

'df.apply(f,axis=1) is 14.2x slower than df.itertuples() on a df of almost 21M rows'

In [68]:
%%time
abbreviations = set()
new_types = []
for row in tqdm(edges.itertuples(), total=len(edges)):
    new_types.append(validate_abbrevs(row[3], row[4], row[5]))

100%|██████████| 20891064/20891064 [01:29<00:00, 232661.06it/s]

CPU times: user 1min 29s, sys: 624 ms, total: 1min 29s
Wall time: 1min 29s


In [69]:
edges = edges.reset_index(drop=True)

In [70]:
e_types_current = edges['TYPE'] + '_' + edges['abbrev']

In [71]:
need_swap = e_types_current != pd.Series(new_types)

In [72]:
tmp = edges.loc[need_swap, 'START_ID']
edges.loc[need_swap, 'START_ID'] = edges.loc[need_swap, 'END_ID']
edges.loc[need_swap, 'END_ID'] = tmp
edges['TYPE'] = new_types

In [73]:
edges = edges[['START_ID', 'END_ID', 'TYPE']]

In [74]:
edges['TYPE'].nunique()

3989

In [75]:
nodes = nodes.dropna()
edges = edges.dropna().drop_duplicates()

In [76]:
print('{:,}'.format(len(nodes)))
print('{:,}'.format(len(edges)))

268,918
20,521,816


In [77]:
edge_ids = set(edges['START_ID']).union(set(edges['END_ID']))
len(edge_ids)

268917

In [78]:
node_ids = set(nodes['ID'])
len(node_ids)

268918

In [79]:
len(edge_ids - node_ids)

0

In [80]:
edge_ids = list(edge_ids)
nodes = nodes.query('ID in @edge_ids')

In [ ]:
nodes['ID'].nunique()

268917

In [ ]:
# Sort values before writing to disk
nodes = nodes.sort_values('LABEL')
edges = edges.sort_values('TYPE')

# Add in colons required by neo4j
nodes.columns = [':ID', 'name', ':LABEL']
edges.columns = [':START_ID', ':END_ID', ':TYPE']

nodes.to_csv('../data/nodes.csv', index=False)
edges.to_csv('../data/edges.csv', index=False)